## Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sage.all import *
from tqdm.auto import tqdm

## Generate RSA keypairs for all public exponents < 47

In [2]:
p_lt_47 = [i for i in primes_first_n(47) if i > 2 and i < 47]

In [3]:
keypairs = list()
for e in tqdm(p_lt_47):
    first_loop = True
    while first_loop or gcd(e, phi) != 1 or N.nbits() != 2048:
        first_loop = False
        # Note: Product of two 1024-bit primes not always 2048 bit
        # As this is just a proof of concept, we just reject if n < 2048 bits.
        p = random_prime(2**1024-1, False, 2**1023)
        q = random_prime(2**1024-1, False, 2**1023)
        N = p * q
        phi = (p - 1) * (q - 1)
    bezout = xgcd(e, phi)
    d = Integer(mod(bezout[1], phi))
    n=N.nbits()
    keypairs.append(dict(p=p, q=q, N=N, n=n, phi=phi, e=e, d=d))
    assert mod(d * e, phi) == 1

  0%|          | 0/13 [00:00<?, ?it/s]

## Utility functions

### Determine the maximum vulnerable padding

In [4]:
for keypair in keypairs:
    keypair['m'] = Integer(ceil(keypair['n'] / keypair['e'] ** 2))

### Function to generate plaintext-padding pairs

In [5]:
def plaintext(N, m):
    n = N.nbits()
    first_loop = True
    while first_loop or M*2**m + 2**m-1 >= N:
        first_loop = False
        M = Integer(randint(2**(n-m-1)-1, 2**(n-m)-1))
    return M

In [6]:
def padding(N, m, M):
    n = N.nbits()
    r = Integer(randint(2**(m-1)-1, 2**m-1))
    M_padded=M*2**m+r
    return dict(M=M, r=r, M_padded=M_padded)

In [7]:
p, q, N, n, phi, e, d, m = keypairs[0].values()
m = 20
M = plaintext(N, m)
M, r, M_padded = padding(N, m, M).values()

In [8]:
power_mod(power_mod(M_padded, e, N), d, N) == M_padded

True

### Function to perform Coppersmith's attack for short padding

In [9]:
def small_roots(self, X=None, beta=1.0, epsilon=None, **kwds):
    r"""
    Let `N` be the characteristic of the base ring this polynomial
    is defined over: ``N = self.base_ring().characteristic()``.
    This method returns small roots of this polynomial modulo some
    factor `b` of `N` with the constraint that `b >= N^\beta`.
    Small in this context means that if `x` is a root of `f`
    modulo `b` then `|x| < X`. This `X` is either provided by the
    user or the maximum `X` is chosen such that this algorithm
    terminates in polynomial time. If `X` is chosen automatically
    it is `X = ceil(1/2 N^{\beta^2/\delta - \epsilon})`.
    The algorithm may also return some roots which are larger than `X`.
    'This algorithm' in this context means Coppersmith's algorithm for finding
    small roots using the LLL algorithm. The implementation of this algorithm
    follows Alexander May's PhD thesis referenced below.

    INPUT:

    - ``X`` -- an absolute bound for the root (default: see above)
    - ``beta`` -- compute a root mod `b` where `b` is a factor of `N` and `b
      \ge N^\beta` (default: 1.0, so `b = N`.)
    - ``epsilon`` -- the parameter `\epsilon` described above. (default: `\beta/8`)
    - ``**kwds`` -- passed through to method :meth:`Matrix_integer_dense.LLL() <sage.matrix.matrix_integer_dense.Matrix_integer_dense.LLL>`

    EXAMPLES:

    First consider a small example::

        sage: N = 10001
        sage: K = Zmod(10001)
        sage: P.<x> = PolynomialRing(K, implementation='NTL')
        sage: f = x^3 + 10*x^2 + 5000*x - 222

    This polynomial has no roots without modular reduction (i.e. over `\ZZ`)::

        sage: f.change_ring(ZZ).roots()
        []

    To compute its roots we need to factor the modulus `N` and use the Chinese
    remainder theorem::

        sage: p, q = N.prime_divisors()
        sage: f.change_ring(GF(p)).roots()
        [(4, 1)]
        sage: f.change_ring(GF(q)).roots()
        [(4, 1)]

        sage: crt(4, 4, p, q)
        4

    This root is quite small compared to `N`, so we can attempt to
    recover it without factoring `N` using Coppersmith's small root
    method::

        sage: f.small_roots()
        [4]

    An application of this method is to consider RSA. We are using 512-bit RSA
    with public exponent `e=3` to encrypt a 56-bit DES key. Because it would be
    easy to attack this setting if no padding was used we pad the key `K` with
    1s to get a large number::

        sage: Nbits, Kbits = 512, 56
        sage: e = 3

    We choose two primes of size 256-bit each::

        sage: p = 2^256 + 2^8 + 2^5 + 2^3 + 1
        sage: q = 2^256 + 2^8 + 2^5 + 2^3 + 2^2 + 1
        sage: N = p*q
        sage: ZmodN = Zmod( N )

    We choose a random key::

        sage: K = ZZ.random_element(0, 2^Kbits)

    and pad it with `512-56=456` 1s::

        sage: Kdigits = K.digits(2)
        sage: M = [0]*Kbits + [1]*(Nbits-Kbits)
        sage: for i in range(len(Kdigits)): M[i] = Kdigits[i]

        sage: M = ZZ(M, 2)

    Now we encrypt the resulting message::

        sage: C = ZmodN(M)^e

    To recover `K` we consider the following polynomial modulo `N`::

        sage: P.<x> = PolynomialRing(ZmodN, implementation='NTL')
        sage: f = (2^Nbits - 2^Kbits + x)^e - C

    and recover its small roots::

        sage: Kbar = f.small_roots()[0]
        sage: K == Kbar
        True

    The same algorithm can be used to factor `N = pq` if partial
    knowledge about `q` is available. This example is from the
    Magma handbook:

    First, we set up `p`, `q` and `N`::

        sage: length = 512
        sage: hidden = 110
        sage: p = next_prime(2^int(round(length/2)))
        sage: q = next_prime(round(pi.n()*p))                                           # needs sage.symbolic
        sage: N = p*q                                                                   # needs sage.symbolic

    Now we disturb the low 110 bits of `q`::

        sage: qbar = q + ZZ.random_element(0, 2^hidden - 1)                             # needs sage.symbolic

    And try to recover `q` from it::

        sage: F.<x> = PolynomialRing(Zmod(N), implementation='NTL')                     # needs sage.symbolic
        sage: f = x - qbar                                                              # needs sage.symbolic

    We know that the error is `\le 2^{\text{hidden}}-1` and that the modulus
    we are looking for is `\ge \sqrt{N}`::

        sage: from sage.misc.verbose import set_verbose
        sage: set_verbose(2)
        sage: d = f.small_roots(X=2^hidden-1, beta=0.5)[0]  # time random               # needs sage.symbolic
        verbose 2 (<module>) m = 4
        verbose 2 (<module>) t = 4
        verbose 2 (<module>) X = 1298074214633706907132624082305023
        verbose 1 (<module>) LLL of 8x8 matrix (algorithm fpLLL:wrapper)
        verbose 1 (<module>) LLL finished (time = 0.006998)
        sage: q == qbar - d                                                             # needs sage.symbolic
        True

    REFERENCES:

    Don Coppersmith. *Finding a small root of a univariate modular equation.*
    In Advances in Cryptology, EuroCrypt 1996, volume 1070 of Lecture
    Notes in Computer Science, p. 155--165. Springer, 1996.
    http://cr.yp.to/bib/2001/coppersmith.pdf

    Alexander May. *New RSA Vulnerabilities Using Lattice Reduction Methods.*
    PhD thesis, University of Paderborn, 2003.
    http://www.cs.uni-paderborn.de/uploads/tx_sibibtex/bp.pdf
    """
    from sage.misc.verbose import verbose
    from sage.matrix.constructor import Matrix
    from sage.rings.real_mpfr import RR

    N = self.parent().characteristic()

    if not self.is_monic():
        raise ArithmeticError("Polynomial must be monic.")

    beta = RR(beta)
    if beta <= 0.0 or beta > 1.0:
        raise ValueError("0.0 < beta <= 1.0 not satisfied.")

    f = self.change_ring(ZZ)

    P,(x,) = f.parent().objgens()

    delta = f.degree()

    if epsilon is None:
        epsilon = beta/8
    verbose("epsilon = %f"%epsilon, level=2)

    m = max(beta**2/(delta * epsilon), 7*beta/delta).ceil()
    verbose("m = %d"%m, level=2)

    t = int( ( delta*m*(1/beta -1) ).floor() )
    verbose("t = %d"%t, level=2)

    if X is None:
        X = (0.5 * N**(beta**2/delta - epsilon)).ceil()
    verbose("X = %s"%X, level=2)

    # we could do this much faster, but this is a cheap step
    # compared to LLL
    g  = [x**j * N**(m-i) * f**i for i in range(m) for j in range(delta) ]
    g.extend([x**i * f**m for i in range(t)]) # h

    B = Matrix(ZZ, len(g), delta*m + max(delta,t) )
    for i in range(B.nrows()):
        for j in range( g[i].degree()+1 ):
            B[i,j] = g[i][j]*X**j

    B =  B.LLL(**kwds)

    f = sum([ZZ(B[0,i]//X**i)*x**i for i in range(B.ncols())])
    R = f.roots()

    ZmodN = self.base_ring()
    roots = set([ZmodN(r) for r,m in R])
    return list(roots)

In [10]:
def coppersmiths_(e, N, C1, C2):
    p_xy_mod_N = Zmod(N)['x', 'y']
    p_y_mod_N = Zmod(N)['y']
    
    x, y = p_xy_mod_N.gens()
    g1 = x ** e - C1
    g2 = (x + y) ** e - C2
    
    g1 = g1.change_ring(p_y_mod_N)
    g2 = g2.change_ring(p_y_mod_N)
    res = g1.resultant(g2, x)
    
    res = res.change_ring(Zmod(N))
    res = res.univariate_polynomial()
    assert res.base_ring().characteristic() == N

    delta = small_roots(res, X=2**(N.nbits()//(2*e*e)), beta=0.5)
    
    #print(diff)
    if len(delta) == 0:
        delta = np.inf
    else:
        delta = delta[0]
        assert(res(delta)) == 0
        return int(delta)
    return delta
def coppersmiths(e, N, C1, C2):
    r2_minus_r1 = coppersmiths_(e, N, C1, C2)
    r1_minus_r2 = coppersmiths_(e, N, C2, C1)
    if r2_minus_r1 > r1_minus_r2:
        r2_minus_r1 = -r1_minus_r2    
    return r2_minus_r1

In [11]:
2**(N.nbits() // (e**2 * 2))

10384593717069655257060992658440192

In [12]:
p, q, N, n, phi, e, d, m = keypairs[0].values()
m = 10

M = plaintext(N, m)
_, r1, M1 = padding(N, m, M).values()
C1 = power_mod(M1, e, N)
_, r2, M2 = padding(N, m, M).values()
C2 = power_mod(M2, e, N)

coppersmiths(e, N, C1, C2), r2 - r1

(387, 387)

## Function to perform the related-message attack

In [13]:
def related_message_attack(N, e, C1, C2, delta):
    x = Zmod(N)['x'].gen()
    g1, g2 = x ** e - C1, (x + delta) ** e - C2
    while g2 != 0:
        g1, g2 = g2, g1 % g2
    polynomial_gcd = g1.monic()
    x_minus_M2 = polynomial_gcd.coefficients()[0]
    M2 = -x_minus_M2
    return M2

In [14]:
p, q, N, n, phi, e, d, m = keypairs[2].values()
m = 3

M = plaintext(N, m)
_, r1, M1 = padding(N, m, M).values()
C1 = power_mod(M1, e, N)
_, r2, M2 = padding(N, m, M).values()
C2 = power_mod(M2, e, N)

delta = coppersmiths(e, N, C1, C2)
print(delta, r2-r1)
related_message_attack(N, e, C1, C2, delta) == M1

1 1


True

## Plot our results

In [15]:
results = []
with tqdm(total=len(keypairs)*300) as pbar:
    for i in range(1):
        for keypair in keypairs:
            e, N = keypair['e'], keypair['N']
            for m in range(1, 228):
                p, q, N, n, phi, e, d, _ = keypair.values()
                if m > n / e**2:
                    pass
    
                M = plaintext(N, m)
                _, r1, M1 = padding(N, m, M).values()
                C1 = power_mod(M1, e, N)
                _, r2, M2 = padding(N, m, M).values()
                C2 = power_mod(M2, e, N)
    
                import time
                # Calculate the start time
                start = time.time()
    
                delta = coppersmiths(e, N, C1, C2)
                if delta < N:
                    if related_message_attack(N, e, C1, C2, delta) == M1:
                        end = time.time()
                        length = end - start
                        results.append(dict(p=p, q=q, N=N, n=n, e=e, d=d, m=m, time=length))
                    else:
                        print(m)
                else:
                    pbar.update(228-m)
                    break
                pbar.update()

TypeError: unsupported format string passed to sage.rings.rational.Rational.__format__

<repr(<tqdm.notebook.TqdmHBox at 0x169994dc0>) failed: TypeError: unsupported format string passed to sage.rings.rational.Rational.__format__>

TypeError: unsupported format string passed to sage.rings.rational.Rational.__format__

## generating keys for wieners attack

In [16]:
e = random_prime(2**16-1, False, 2**15)
first_loop = True
while first_loop or gcd(e, phi) != 1 or N.nbits() != 2048:
    first_loop = False
    # Note: Product of two 1024-bit primes not always 2048 bit
    # As this is just a proof of concept, we just reject if n < 2048 bits.
    p = random_prime(2**1024-1, False, 2**1023)
    q = random_prime(2**1024-1, False, 2**1023)
    N = p * q
    phi = (p - 1) * (q - 1)
bezout = xgcd(e, phi)
d = Integer(mod(bezout[1], phi))
n=N.nbits()
keypairs.append(dict(p=p, q=q, N=N, n=n, phi=phi, e=e, d=d))
assert mod(d * e, phi) == 1
# swap the public and private exponent
e, d = d, e
# check that now we have small private exponent, big public exponent, and still e coprime to phi
d.nbits(), e.nbits(), gcd(e, phi)

(16, 2047, 1)